# Indicadores DIT

In [123]:
#Librerías
import pandas as pd
import os
import numpy as np

In [49]:
os.chdir(r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\03. ENDES\1. Data')

# Modulo DIT
dit = pd.DataFrame()
for anio in range(2019,2023):
    df = pd.read_stata(f"dit_{anio}.dta", convert_categoricals=True)
    df.columns = df.columns.str.lower()
    dit= pd.concat([dit,df], ignore_index=True)
dit['id1'] = dit['id1'].astype(int)

# Modulo REC21
rec21 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['ID1', 'CASEID', 'BIDX', 'B4'])
    except:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['id1', 'caseid', 'bidx', 'b4'])
    df.columns = df.columns.str.lower()
    rec21 = pd.concat([rec21,df], ignore_index=True)

# Modulo REC0111
rec0111 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['ID1', 'CASEID', 'V001', 'V005', 'V012', 'V022' ,'V024' ,'V025' ,'V149' ,'V190'])
    except:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['id1','caseid', 'v001', 'v005', 'v012', 'v022' ,'v024' ,'v025' ,'v149' ,'v190'])
    df.columns = df.columns.str.lower()
    rec0111= pd.concat([rec0111,df], ignore_index=True)

# Modulo REC91
rec91 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
    except:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rec91 = pd.concat([rec91, df], ignore_index=True)

C:\Users\User\AppData\Local\Temp\ipykernel_34360\266190699.py:35: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
C:\Users\User\AppData\Local\Temp\ipykernel_34360\266190699.py:37: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
C:\Users\User\AppData\Local\Temp\ipyke

In [64]:
# Merging datasets
mergedDIT = pd.merge(dit, rec21.loc[:,['id1','caseid', 'bidx', 'b4']], how='inner', on=['id1','caseid', 'bidx'], validate="1:1")
mergedDIT = pd.merge(mergedDIT, rec0111.loc[:,['id1','caseid', 'v001', 'v005', 'v012', 'v022' ,'v024' ,'v025' ,'v149' ,'v190']], how='inner', on=['id1','caseid'], validate="m:1")
mergedDIT = pd.merge(mergedDIT, rec91.loc[:,['id1','caseid','sregion', 's119', 's108n']], how='inner', on=['id1','caseid'], validate="m:1")

In [76]:
# Generando variables de características del hogar/madre
mergedDIT['v149_'] = mergedDIT['v149'].array.codes
mergedDIT['educa'] = 1  # Initialize with a default value
mergedDIT.loc[mergedDIT['v149_'] > 2, 'educa'] = 2
mergedDIT.loc[mergedDIT['v149_'] == 5, 'educa'] = 3
del mergedDIT['v149_']

### Generación de variables de apego

In [151]:
## Porcentaje de madres de niñas y niños entre 9 y 12 meses de edad que responde de manera inmediata y adecuada al llanto de su hija/o
mergedDIT['sensMadre9a12'] = 0
mergedDIT['qi478'] = mergedDIT['qi478'].values
cond = (mergedDIT['qi478'] >= 9) & (mergedDIT['qi478'] <= 12)
cond1 = (mergedDIT['qi478e10'] == "Deja de hacer lo que está haciendo y la/lo atiende") | (mergedDIT['qi478e10'] ==  "Le pide otra persona que la(o) atienda") | (mergedDIT['qi478e10'] == "La (Lo) atiende aun cuando siga haciendo sus labores")
mergedDIT.loc[ cond & cond1 ,'sensMadre9a12'] = 1
mergedDIT.loc[-cond,'sensMadre9a12'] = np.nan

## Porcentaje de niños y niñas entre 9 y 12 meses de edad que tiene adecuada interacción madre-hijo/a como precursor del apego seguro
mergedDIT['apego9a12'] = 0
mergedDIT['apego9a12a'] = 0
mergedDIT['apego9a12b'] = 0
mergedDIT['apego9a12c'] = 0
cond1 = (mergedDIT['qi478e7'] == "No") 
cond2 = (mergedDIT['qi478e8'] == "No") 
cond3 = (mergedDIT['qi478e9'] == "No")

# Apego seguro - dummy
mergedDIT.loc[ cond & cond1 & cond2 & cond3 ,'apego9a12'] = 1
mergedDIT.loc[-cond,'apego9a12'] = np.nan

# Apego seguro - categórica
mergedDIT.loc[ cond & cond1 ,'apego9a12a'] = 1
mergedDIT.loc[-cond,'apego9a12a'] = np.nan
mergedDIT.loc[ cond & cond2 ,'apego9a12b'] = 1
mergedDIT.loc[-cond,'apego9a12b'] = np.nan
mergedDIT.loc[ cond & cond3 ,'apego9a12c'] = 1
mergedDIT.loc[-cond,'apego9a12c'] = np.nan
mergedDIT['apego9a12_cat'] = mergedDIT['apego9a12a'] + mergedDIT['apego9a12b'] + mergedDIT['apego9a12c']
mergedDIT.loc[-cond,'apego9a12_cat'] = np.nan

del mergedDIT['apego9a12a'] , mergedDIT['apego9a12b'] , mergedDIT['apego9a12c']

In [134]:
mergedDIT['apego9a12_cat'].value_counts()

apego9a12_cat
3.0    2331
2.0    1841
1.0     678
0.0     399
Name: count, dtype: int64

### Generando variables de comunicación efectiva

In [150]:
#Porcentaje de niñas y niños de 9 a 36 meses con comunicación verbal a nivel comprensivo y expresivo apropiada para su edad
for grupo in ['9a12','13a18', '19a23', '24a36']:
    mergedDIT[f'comEfec{grupo}'] = 0
    mergedDIT[f'comEfec{grupo}a'] = 0
    mergedDIT[f'comEfec{grupo}b'] = 0
    mergedDIT[f'comEfec{grupo}c'] = 0


mergedDIT['qi478'] = mergedDIT['qi478'].values
cond9a12 = (mergedDIT['qi478'] >= 9) & (mergedDIT['qi478'] <= 12) 
cond9a12a = (mergedDIT['qi478e3'] == "Si")
cond9a12b = (mergedDIT['qi478e4'] == "Si")
cond9a12c = (mergedDIT['qi478e5'] == "Si")
cond13a18 = (mergedDIT['qi478'] >= 13) & (mergedDIT['qi478'] <= 18)
cond13a18a = (mergedDIT['qi478f3'] == "Si")
cond13a18b = (mergedDIT['qi478f4'] == "Si")
cond13a18c = (mergedDIT['qi478f5'] == "Si")
cond19a23 = (mergedDIT['qi478'] >= 19) & (mergedDIT['qi478'] <= 23)
cond19a23a = (mergedDIT['qi478g1'] == "Si")
cond19a23b = ((mergedDIT['qi478g2_a'] == "Si") & (mergedDIT['qi478g2_b'] == "Si") & (mergedDIT['qi478g2_c'] == "Si"))
cond19a23c = (mergedDIT['qi478g3'] == "Si")
cond24a36 = (mergedDIT['qi478'] >= 24) & (mergedDIT['qi478'] <= 36)
cond24a36a = (mergedDIT['qi478h1'] == "Si")
cond24a36b = (mergedDIT['qi478h2'] == "Si") 
cond24a36c = (mergedDIT['qi478h3'] == "Si")

for grupo in ['9a12','13a18', '19a23', '24a36']:
    # Comunicacion efectiva - dummy por edad
    mergedDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}a'] & locals()[f'cond{grupo}b'] & locals()[f'cond{grupo}c'] ,f'comEfec{grupo}'] = 1
    mergedDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}'] = np.nan

    # Comunicacion efectiva - categórica por edad
    mergedDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}a'] ,f'comEfec{grupo}a'] = 1
    mergedDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}a'] = np.nan
    mergedDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}b'] ,f'comEfec{grupo}b'] = 1
    mergedDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}b'] = np.nan
    mergedDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}c'] ,f'comEfec{grupo}c'] = 1
    mergedDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}c'] = np.nan
    mergedDIT[f'comEfec{grupo}_cat'] = mergedDIT[f'comEfec{grupo}a'] + mergedDIT[f'comEfec{grupo}b'] + mergedDIT[f'comEfec{grupo}c']
    mergedDIT.loc[-locals()[f'cond{grupo}'], f'comEfec{grupo}_cat'] = np.nan
    
    del mergedDIT[f'comEfec{grupo}a'] , mergedDIT[f'comEfec{grupo}b'] , mergedDIT[f'comEfec{grupo}c']


for var_name in list(locals()):
    if var_name.startswith('cond'):
        del locals()[var_name]

In [153]:
correlation_matrix = mergedDIT[['apego9a12', 'apego9a12_cat', 'comEfec9a12', 'comEfec9a12_cat']].corr()
correlation_matrix

,apego9a12,apego9a12_cat,comEfec9a12,comEfec9a12_cat
apego9a12,1.000000,0.811155,-0.026716,-0.007959
apego9a12_cat,0.811155,1.000000,0.002029,0.047939
comEfec9a12,-0.026716,0.002029,1.000000,0.881844
comEfec9a12_cat,-0.007959,0.047939,0.881844,1.000000


In [149]:
mergedDIT['comEfec9a12_cat'].value_counts()

comEfec9a12_cat
3.0    4014
2.0     895
1.0     232
0.0     108
Name: count, dtype: int64

In [126]:
mergedDIT['qi478e7'].value_counts()

qi478e7
No                     3947
Si                     1255
No responde/No sabe       2
Name: count, dtype: int64

In [128]:
mergedDIT['precursorApego9a12'].value_counts()

precursorApego9a12
0.0    2918
1.0    2331
Name: count, dtype: int64

In [ ]:
# Factor de expansion
merged_final["wt"] = merged_final["v005"] / 1000000

# Generamos la variable EDADM
merged_final["edadm"] = merged_final["v008"] - merged_final["b3"]
merged_final["e_0a35"] = merged_final["edadm"] < 36
merged_final["e_0a59"] = merged_final["edadm"] < 60

# Peso al nacer
merged_final["pesonac"] = 1
merged_final.loc[(merged_final["m19"] > 2499) & (merged_final["m19"] <= 8000) & (merged_final["v012"] > 14), "pesonac"] = 2
merged_final.loc[merged_final["m19"] == 9996 & (merged_final["v012"] > 14), "pesonac"] = 3
merged_final.loc[merged_final["m19"] == 9998 & (merged_final["v012"] > 14), "pesonac"] = 4

# Labels for pesonac
labels = {1: "Menos de 2.5kg", 2: ">=2.5 kg", 3: "No pesados al nacer", 4: "No sabe, sin informacion"}
merged_final["pesonac"] = merged_final["pesonac"].map(labels)

In [ ]:
# Tabulations
tab1 = pd.crosstab(merged_final["pesonac"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab2 = pd.crosstab([merged_final["v463a"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab3 = pd.crosstab([merged_final["v190"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab4 = pd.crosstab([merged_final["etnia"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab5 = pd.crosstab([merged_final["lmaterna"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab6 = pd.crosstab([merged_final["v025"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab7 = pd.crosstab([merged_final["sregion"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")

# Tamaño de la niña o niño al nacer
merged_final["ch_size_birth"] = pd.cut(merged_final["m18"], bins=[-1, 2, 4, 5, 8, 9], labels=["Promedio o mayor", "Menor que promedio", "Muy pequeño", "No sabe/sin información"], right=False)
tab8 = pd.crosstab(merged_final["ch_size_birth"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab9 = pd.crosstab([merged_final["v463a"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab10 = pd.crosstab([merged_final["v190"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab11 = pd.crosstab([merged_final["etnia"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab12 = pd.crosstab([merged_final["lmaterna"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab13 = pd.crosstab([merged_final["v025"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab14 = pd.crosstab([merged_final["sregion"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
